<a href="https://colab.research.google.com/github/JuanFelipePosada/python-basico_/blob/main/punto_1_parcial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install -U yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy
!pip install control

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Usi

In [23]:
%%writefile app.py
import streamlit as st
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import control as ctl


# Definir variables simbólicas
s = sp.symbols('s')
m = sp.symbols('m')  # Masa como variable
c_value = 1.0  # Valor fijo para el coeficiente de amortiguamiento
k_value = 1.0  # Valor fijo para la constante del resorte
wn, zeta = sp.symbols('omega_n zeta')  # Frecuencia natural y factor de amortiguamiento

# Título de la aplicación
st.title("Proceso para hallar la función de transferencia")

# Descripción inicial
st.write("""
El objetivo es encontrar la función de transferencia del sistema masa-resorte-amortiguador.
La ecuación de movimiento es:
""")

# Mostrar la ecuación diferencial
st.latex(r"m \frac{d^2x(t)}{dt^2} + c \frac{dx(t)}{dt} + k x(t) = Y(t)")

# Paso 1: Transformada de Laplace
st.subheader("Paso 1: Transformada de Laplace")
st.write("Aplicamos la transformada de Laplace a la ecuación diferencial, suponiendo condiciones iniciales cero:")
st.latex(r"m s^2 X(s) + c s X(s) + k X(s) = Y(s)")

# Paso 2: Despejar X(s)
st.subheader("Paso 2: Despejar X(s)")
st.write("Despejamos \( X(s) \) en términos de \( Y(s) \):")
X_s = sp.symbols('X_s')
Y_s = sp.symbols('Y_s')
eq = sp.Eq(m * s**2 * X_s + c_value * s * X_s + k_value * X_s, Y_s)
st.latex(sp.latex(eq))

# Paso 3: Función de transferencia
st.subheader("Paso 3: Función de transferencia")
st.write("""
La función de transferencia \( H(s) \) se define como la relación entre la salida \( X(s) \) y la entrada \( Y(s) \).
Despejamos \( H(s) \):
""")
H_s = sp.simplify(Y_s / (m * s**2 + c_value * s + k_value))
st.latex(r"H(s) = \frac{X(s)}{Y(s)} = \frac{1}{ms^2 + cs + k}")

# Paso 4: Interpretación
st.subheader("Paso 4: Interpretación")
st.write("""
Esta es la función de transferencia del sistema masa-resorte-amortiguador.
Describe cómo la salida (el desplazamiento \( X(s) \)) responde a una entrada (la fuerza \( Y(s) \)).
""")
#
# Mostrar la función de transferencia final
st.write("La función de transferencia es:")
st.latex(sp.latex(1 / (m * s**2 + c_value * s + k_value)))

# Añadiendo la parte de los coeficientes y frecuencia natural
st.subheader("Coeficientes y Reescritura en términos de \( omega_n ) y \( chi )")

# Entradas para valores de m
st.write("Ingrese el valor de la masa (m):")
m_value = st.number_input("Masa (m)", value=1.0)

# Calcular frecuencia natural wn y factor de amortiguamiento zeta
wn_value = sp.sqrt(k_value / m_value)
zeta_value = c_value / (2 * sp.sqrt(k_value * m_value))

# Mostrar resultados de wn y zeta
st.write("Resultados:")
st.latex(r"\omega_n = \sqrt{\frac{k}{m}} = \sqrt{\frac{" + str(k_value) + "}{" + str(m_value) + "}} = " + str(wn_value.evalf()))
st.latex(r"\zeta = \frac{c}{2\sqrt{km}} = \frac{" + str(c_value) + "}{2\sqrt{" + str(k_value) + " \cdot " + str(m_value) + "}} = " + str(zeta_value.evalf()))

# Función de transferencia en términos de wn y chi
st.write("""
La función de transferencia también se puede escribir en términos de la frecuencia natural \( omega_n ) y el factor de amortiguamiento \( chi ):
""")
H_s = k_value * (wn**2 / (s**2 + 2 * zeta * wn * s + wn**2))
st.latex(r"H(s) = k \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}")

st.write("Esta es la forma final en términos de \( omega_n ) y \( chi ).")
########################################################################################################################################################


# Definir variables simbólicas para el modelado
s = sp.symbols('s')
wn, zeta = sp.symbols('omega_n zeta')  # Frecuencia natural y factor de amortiguamiento

# Título del dashboard
st.title("Simulación del Sistema Masa-Resorte-Amortiguador")

# Descripción inicial
st.write("""
Este dashboard permite simular un sistema masa-resorte-amortiguador bajo diferentes condiciones de amortiguamiento.
El usuario puede ajustar el valor del factor de amortiguamiento \( \zeta \) y visualizar la respuesta del sistema.
""")

# Entrada del factor de amortiguamiento
zeta_value = st.slider('Seleccione el valor del factor de amortiguamiento (zeta)', 0.0, 2.0, 0.5, step=0.1)

# Calcular parámetros del sistema en función de zeta y wn
wn_value = 1.0  # Frecuencia natural fija
c_value = 2 * zeta_value * wn_value
m_value = 1.0  # Masa fija
k_value = wn_value ** 2 * m_value

# Equivalentes eléctricos
R_value = c_value
L_value = m_value
C_value = 1 / k_value

# Mostrar los valores de los parámetros calculados
st.write("Valores calculados:")
st.write(f"Coeficiente de amortiguamiento (c): {c_value}")
st.write(f"Constante del resorte (k): {k_value}")
st.write(f"Masa (m): {m_value}")
st.write("Equivalentes eléctricos:")
st.write(f"Resistencia (R): {R_value}")
st.write(f"Inductancia (L): {L_value}")
st.write(f"Capacitancia (C): {C_value}")

# Determinar el tipo de amortiguamiento
if zeta_value < 1:
    damping_type = "Subamortiguado (Oscilatorio)"
elif zeta_value == 1:
    damping_type = "Amortiguamiento crítico"
elif 1 < zeta_value < 2:
    damping_type = "Sobreamortiguado"
else:
    damping_type = "Inestable"

st.write(f"El sistema es: {damping_type}")

# Función de transferencia del sistema en función de wn y zeta
num = [wn_value ** 2]  # Numerador
den = [1, 2 * zeta_value * wn_value, wn_value ** 2]  # Denominador
H_s = ctl.TransferFunction(num, den)

# Mostrar la función de transferencia
st.write("La función de transferencia del sistema es:")
st.latex(r"H(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}")

# Gráficos de simulación
st.subheader("Gráficas de Simulación")

# Paso 1: Gráfica de polos y ceros
st.write("Polos y ceros:")
plt.figure()
ctl.pzmap(H_s, plot=True, grid=True)
st.pyplot(plt)

# Paso 2: Diagrama de Bode
st.write("Diagrama de Bode:")
plt.figure()
mag, phase, omega = ctl.bode(H_s, dB=True, Hz=False, deg=True, plot=True)
st.pyplot(plt)

# Paso 3: Respuesta al impulso
st.write("Respuesta al impulso:")
plt.figure()
T, yout = ctl.impulse_response(H_s)
plt.plot(T, yout)
plt.title('Respuesta al Impulso')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.grid(True)
st.pyplot(plt)

# Paso 4: Respuesta al escalón
st.write("Respuesta al escalón:")
plt.figure()
T, yout = ctl.step_response(H_s)
plt.plot(T, yout)
plt.title('Respuesta al Escalón')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.grid(True)
st.pyplot(plt)


Overwriting app.py


In [24]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFiUsTGsQClQyxWz9mY9rLIup_7vUusFxPVHjfdMEa1CryT"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://92a6-34-86-137-114.ngrok-free.app" -> "http://localhost:5011"
